# Avaliação de Redes Complexas

**Nome:** Luiz Fernando Rabelo (11796893)

<hr>

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import EoN
import pandas as pd
from community import community_louvain
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

**1 -** (a) Crie o gráfo a seguir usando a biblioteca networkx e mostre o gráfo.<br>
Link para visualizar o grafo: https://commons.wikimedia.org/wiki/File:6n-graf.png<br>

In [ ]:
# Criação da lista de vértices:
vertices = [1, 2, 3, 4, 5, 6]

# Criação da lista de arestas:
arestas = [(1,2), (1,5), (2,3), (2,5), (3,4), (4,5), (4,6)]

# Geração do grafo a partir dos vértices e arestas criados:
G = nx.Graph()
G.add_nodes_from(vertices)
G.add_edges_from(arestas)

# Desenho do grafo:
pos = nx.spring_layout(G)
nx.draw(G, pos=pos, with_labels=True)

(b) Simule uma caminha aleatória nesse grafo e monte uma matriz onde o elemento Mij representa o número de visitas ao vértice j dado que a caminhada iniciou em i. Considere pelo menos 100 passos. Compare o número de visitas com a medida eigenvector centrality de cada vértice.

In [ ]:
# Inicialização de variáveis:
n = len(G)
total_passos = 3000
matriz_visitas = np.zeros((n, n))
matriz_adjacencia = nx.to_numpy_array(G)
matriz_transicao = matriz_adjacencia / matriz_adjacencia.sum(axis=1)[:,None]

# Caminhada aleatória:
for vertice_inicial in range(1, n+1):
    vertice_atual = vertice_inicial
    for contagem_passos in range(total_passos):
        matriz_visitas[vertice_inicial-1][vertice_atual-1] += 1
        vertice_atual = np.random.choice(G.nodes, p=matriz_transicao[vertice_atual-1])

# Cálculo da eigenvector centrality:
eigen_centrality = np.array(list(nx.eigenvector_centrality(G, max_iter=total_passos).values()))

# Normalização da matriz de visitas e do eigenvector:
matriz_visitas /= total_passos
eigen_centrality /= eigen_centrality.sum()

# Cálculo da média de visitas na matriz computada na caminhada aleatória:
media_visitas = [np.mean(matriz_visitas[:,i]) for i in range(n)]

# Desenho gráfico dos resultados:
plt.figure(figsize=(10,6))
plt.title('Comparação Caminhadas Aleatórias vs Medida Eigenvector Centrality')
plt.xlabel('Vértices')
plt.ylabel('% Visitas')
plt.plot(G.nodes, eigen_centrality, 'o-', ms=9, label='Eigenvector Centrality')
for i in range(n):
    plt.plot(G.nodes, matriz_visitas[i], 'o', label=f'Caminhada Aleatória início={i+1}')
plt.plot(G.nodes, media_visitas, 'o-', ms=9, label='Média Caminhadas Aleatórias')
plt.legend(loc=(1.01, 0));

<hr>

**2 -** (a) Leia a rede do arquivo advogato.txt e mostre a distribuição acumulada complementar do grau, isto é, $P(K > k)$.

In [ ]:
# Leitura da rede advogato:
G = nx.read_edgelist('data/advogato.txt')

# Seleção do maior componente conectado:
G = G.subgraph(sorted(nx.connected_components(G), key=len, reverse=True)[0])

# Transformação dos labels de strings para inteiros:
G = nx.convert_node_labels_to_integers(G)

# Determinação da distribuição do grau:
graus = np.array(list(dict(G.degree()).values()))
grau_maximo = np.max(graus)
ocorrencias = np.zeros(grau_maximo + 1)
for grau in graus:
    ocorrencias[grau] += 1

# Determinação da distribuição acumulada complementar do grau:
dist_acc_complementar = np.zeros(grau_maximo + 1)
dist_acc_complementar[0] = len(graus) - ocorrencias[0]
for i in range(1, grau_maximo + 1):
    dist_acc_complementar[i] = len(graus) - ocorrencias[i] - dist_acc_complementar[i-1]

# Representação do resultado:
plt.figure(figsize=(10,6))
plt.title('Distribuição Acumulada Complementar do Grau')
fig = plt.subplot(1, 1, 1)
fig.set_xscale('log')
fig.set_yscale('log')
plt.xlabel('k')
plt.ylabel('Pa(k)')
plt.plot(ocorrencias, 'o', ms=5, mec='black');

(b) Mostre o gráfico de Knn(k), ou seja, o grau médio dos vizinhos dos vértices de grau k em função do grau k.

In [ ]:
# Determinação da lista de graus médios da vizinhança:
graus_medios_vizinhanca = np.array([
    float(nx.average_neighbor_degree(G, nodes=[vertice])[vertice]) for vertice in G.nodes
])

# Determinação da lista de graus de todos os vértices: 
graus_vertices = list(dict(G.degree()).values())

# Inicialização das listas de vizinhos e de graus médios:
graus_vizinhos = []
clusterings_medios = []

# Preenchimento da lista de vizinhos e de graus médios:
for grau in np.arange(np.min(graus_vertices), np.max(graus_vertices) + 1):
    grau_atual = graus_vertices == grau
    if len(graus_medios_vizinhanca[grau_atual]) > 0:
        graus_vizinhos.append(grau)
        clusterings_medios.append(np.mean(graus_medios_vizinhanca[grau_atual]))

# Desenho de knn(k) em função de k:
fig = plt.figure(figsize=(10,6))
plt.title('Grau Médio dos Vizinhos dos Vértices de Grau k')
plt.xlabel('k')
plt.ylabel('knn(k)')
plt.plot(graus_vizinhos, clusterings_medios, 'o', ms=5, mec='black', label='Grau médio dos vizinhos')

# Desenho da melhor reta de aproximação dos pontos plotados:
coeficientes = np.polyfit(graus_vizinhos, clusterings_medios, deg=1, full=True)
a, b = coeficientes[0][0], coeficientes[0][1]
xs = [min(graus_vizinhos), max(graus_vizinhos)]
ys = [a*x + b for x in xs]
plt.plot(xs, ys, '--', lw=2, label=f'Aproximação {round(a, 2)}x+{round(b,3)}');
plt.legend();

(c) Calcule a matriz de menores distâncias $D$ a matriz $A^n$, que presenta o número de caminhadas de comprimento $n$ entre cada par de vértices. Faça um gráfico da correlação entre $D_{ij}$ e $A_{ij}$ para diferentes valores de $n$. Ou seja, um gráfico em que o eixo x representa n e o y, a correlação entre essas duas matrizes.

In [ ]:
# Inicialização da matriz de distâncias:
n = len(G)
distancias = np.zeros((n,n))

# Preenchimento da matriz de distâncias:
for i in range(n):
    for j in range(i+1, n):
        if i != j:
            distancias[i][j] = distancias[j][i] = len(nx.shortest_path(G, i, j)) - 1

# Determinação da matriz A de adjacências:
matriz_adjacencia = nx.to_numpy_array(G)

# Deteminação do número de caminhadas aleatórias a partir da matriz A:
comprimento_maximo = 10
matrizes_caminhadas = [matriz_adjacencia]
for _ in range(1, comprimento_maximo):
    matrizes_caminhadas.append(matrizes_caminhadas[-1] @ matriz_adjacencia)

# Cálculo das correlações de Pearson entre a distância e caminhadas:
correlacoes = []
for i in range(comprimento_maximo):
    correlacoes.append(np.corrcoef(distancias.flatten(), matrizes_caminhadas[i].flatten())[0,1])

# Desenho do resultado:
xs = [i+1 for i in range(comprimento_maximo)]
plt.figure(figsize=(10,6))
plt.title('Correlação entre Menores Distâncias e Número de Caminhadas de Comprimento n')
plt.xlabel('n')
plt.ylabel('ρ(Dij, Aij^n)')
plt.xticks(xs)
plt.plot(xs, correlacoes, 'o-');

<hr>

**3 -** Considere a rede do arquivo internet_routers-22july06.gml. Escreva um código que calcule as seguintes medidas: (a) grau médio, (b) transitividade, (c) segundo momento da distribuição do grau, (d) entropia de Shannon da distribuição do grau, (e) coeficiente de assortatividade. Considere apenas o maior componente. Armazene os valoes em uma lista e imprima essa lista, indicando os valores de cada medida. 

In [ ]:
# Leitura da rede internet routers:
G = nx.read_gml('data/internet_routers-22july06.gml')

# Seleção do maior componente conectado:
G = G.subgraph(sorted(nx.connected_components(G), key=len, reverse=True)[0])

# Transformação dos labels de strings para inteiros:
G = nx.convert_node_labels_to_integers(G)

# Cálculo do grau médio:
grau_medio = np.mean(np.array(list(dict(G.degree()).values())))

# Cálculo da transitividade:
transitividade = nx.transitivity(G)

# Cálculo do segundo momento da distribuição do grau:
segundo_momento = 0
for vertice in G.nodes:
    segundo_momento += G.degree(vertice) ** 2
segundo_momento /= len(G)

# Cálculo da entropia de Shannon da distribuição do grau:
graus = np.array(list(dict(G.degree()).values()))
grau_maximo = np.max(graus)
ocorrencias = np.zeros(grau_maximo + 1)
for grau in graus:
    ocorrencias[grau] += 1
distribuicao_normalizada = ocorrencias / sum(ocorrencias)
entropia = -sum([p * np.log2(p) for p in distribuicao_normalizada if p > 0])

# Cálculo do coeficiente de assortatividade:
assortatividade = nx.degree_assortativity_coefficient(G)

# Mostragem da lista de resultados:
print('- Grau médio =', grau_medio)
print('- Transitividade =', transitividade)
print('- Segundo momento do grau =', segundo_momento)
print('- Entropia de Shannon do grau =', entropia)
print('- Coeficiente de assortatividade =', assortatividade)

<hr>

**4 -** Simule o modelo de propagação de epidemias SIR na rede de aeroportos dos EUA (USairport_2010.txt). Calcule a correlação de Pearson entre a fração final de recuperados e as seguintes medidas de centralidade: (a) grau, (b) betweenness centrality, (c) eigenvector centrality, (d) closenness centrality. Mostre os resultados e indique a maior correlação. Considere $\beta = 0.06$ e $\mu = 0.1$ no modelo SIR.

In [ ]:
# Leitura da rede us airport:
G = nx.read_weighted_edgelist('data/usairport.txt')

# Seleção do maior componente conectado:
G = G.subgraph(sorted(nx.connected_components(G), key=len, reverse=True)[0])

# Transformação dos labels de strings para inteiros:
G = nx.convert_node_labels_to_integers(G)

# Definição das taxas de propagação beta e mu:
beta = 0.06   # contágio
mu = 0.1      # recuperação

# Definição do número de experimentos por vértice semente:
total_experimentos = 5

# Inicialização de variáveis complementares:
fracoes_sucetiveis = []
fracoes_infectados = []
fracoes_recuperados = []
recuperados_finais_por_vertice = []
tempo_maximo = -1
n = len(G)

# Propagação da epidemia:
for vertice in G.nodes:
    recuperados_vertice = []
    for _ in range(total_experimentos):
        tempos, S, I, R = EoN.fast_SIR(G, beta, mu, initial_infecteds=vertice)
        recuperados_vertice.append(R[-1])
        tempo_maximo = max(tempo_maximo, len(tempos))
        fracoes_sucetiveis.append(S / n)
        fracoes_infectados.append(I / n)
        fracoes_recuperados.append(R / n)
    recuperados_finais_por_vertice.append(np.mean(recuperados_vertice))

# Cálculo das médias percentuais de sucetíveis:
medias_sucetiveis = np.zeros(tempo_maximo)
for i in range(len(fracoes_sucetiveis)):
    for j in range(len(fracoes_sucetiveis[i])):
        medias_sucetiveis[j] = medias_sucetiveis[j] + fracoes_sucetiveis[i][j]
medias_sucetiveis /= len(fracoes_sucetiveis)

# Cálculo das médias percentuais de infectados:
medias_infectados = np.zeros(tempo_maximo)
for i in range(len(fracoes_infectados)):
    for j in range(len(fracoes_infectados[i])):
        medias_infectados[j] += fracoes_infectados[i][j]
medias_infectados /= len(fracoes_infectados)

# Cálculo das médias percentuais de recuperados:
medias_recuperados = np.zeros(tempo_maximo)
for i in range(len(fracoes_recuperados)):
    for j in range(len(fracoes_recuperados[i])):
        medias_recuperados[j] += fracoes_recuperados[i][j]
medias_recuperados /= len(fracoes_recuperados)

# Normalização do array de recuperados finais:
recuperados_finais_por_vertice = np.array(recuperados_finais_por_vertice) / n
print('Percentual médio de recuperados finais:', np.mean(recuperados_finais_por_vertice))

# Desenho da evolução da epidemia:
plt.figure(figsize=(10,6))
plt.title('Evolução Modelo SIR (médias n sementes)')
plt.xlabel('t')
plt.ylabel('% vértices')
plt.plot(range(tempo_maximo), medias_sucetiveis, '-o', label='Vértices Sucetíveis')
plt.plot(range(tempo_maximo), medias_infectados, '-o', label='Vértices Infectados')
plt.plot(range(tempo_maximo), medias_recuperados, '-o', label='Vértices Recuperados')
plt.legend();

In [ ]:
# Determinação das medidas de centralidade:
medidas_centralidade = [
    list(dict(G.degree()).values()),                            # grau
    list(dict(nx.betweenness_centrality(G)).values()),          # betweeness centrality
    list(nx.eigenvector_centrality(G, max_iter=2000).values()), # eigenvector centrality
    list(nx.closeness_centrality(G).values())                   # closeness centrality
]

# Determinação da lista dos rótulos de cada medida:
labels_medidas = ['Grau', 'Betweeness Centrality', 'Eigenvector Centrality', 'Closeness Centrality']

# Cálculo e mostragem das correlações:
plt.figure(figsize=(10,6))
plt.title('Correlações entre Recuperados Finais e Medidas de Centralidade')
plt.xlabel('Medidas de Centralidade')
plt.ylabel('Correlação com Recuperados Finais')
plt.xticks([i for i in range(len(medidas_centralidade))], labels_medidas)
for i in range(len(medidas_centralidade)):
    correlacao = np.corrcoef(recuperados_finais_por_vertice, medidas_centralidade[i])[0,1]
    print(f'- Correlação entre Fração de Recuperados Finais e {labels_medidas[i]} = {correlacao}')
    plt.plot([i], [correlacao], 'P', ms='15')

<hr>

**5 -** Gere redes do tipo BA, ER e WS (p=0.05) com grau médio igual a 10 e N = 500. Desenvolva um estudo para mostrar que essas apresentam topologias diferentes. (Seja criativ@).

In [ ]:
# Definição dos parâmetros das redes:
n = 500
grau_medio = 10
p = 0.05
total_experimentos = 50

# Inicialização de dicionário de dados:
dados_coletados = {
    tipo_rede: {
        'primeiro_momento': [],
        'segundo_momento': [],
        'clustering_medio': [],
        'modularidade': [],
        'media_menor_caminho': [],
        'assortatividade_grau': [],
    }
    for tipo_rede in ['BA', 'ER', 'WS']
}

# Execução de experimentos:
for _ in range(total_experimentos):

    # Geração das redes:
    redes = {
        'BA': nx.barabasi_albert_graph(n, grau_medio),
        'ER': nx.erdos_renyi_graph(n, p),
        'WS': nx.watts_strogatz_graph(n, grau_medio, p)
    }
    
    # Cálculo das features:
    for tipo_rede in redes.keys():
        G = redes[tipo_rede]
        
        # Determinação do primeiro momento do grau:
        soma = 0
        for node in G.nodes:
            soma += G.degree(node)
        dados_coletados[tipo_rede]['primeiro_momento'].append(soma / n)

        # Determinação do segundo momento do grau:
        soma = 0
        for node in G.nodes:
            soma += G.degree(node) ** 2
        dados_coletados[tipo_rede]['segundo_momento'].append(soma / n)

        # Determinação do average clustering:
        clustering_medio = nx.average_clustering(G)
        dados_coletados[tipo_rede]['clustering_medio'].append(clustering_medio)

        # Determinação da modularidade:
        particao = list(community_louvain.best_partition(G).values())
        matriz_adjacencia = nx.adjacency_matrix(G)
        total_arestas = G.number_of_edges()
        soma = 0
        for i in np.arange(n):
            ki = len(list(G.neighbors(i)))
            for j in np.arange(n):
                if particao[i] == particao[j]:
                    kj = len(list(G.neighbors(j)))
                    soma += matriz_adjacencia[i,j] - (ki * kj) / (2 * total_arestas)
        dados_coletados[tipo_rede]['modularidade'].append(soma / (2 * total_arestas))

        # Determinação do menor caminho médio:
        caminho_medio = nx.average_shortest_path_length(G)
        dados_coletados[tipo_rede]['media_menor_caminho'].append(caminho_medio)

        # Determinação do coeficiente de assortatividade do grau:
        assortatividade = nx.degree_assortativity_coefficient(G)
        dados_coletados[tipo_rede]['assortatividade_grau'].append(assortatividade)

# Criação de dataframes para cada tipo de rede:
df_ba = pd.DataFrame.from_dict(dados_coletados['BA'])
print('Resultados Experimentos em Rede Barabasi Albert')
display(df_ba)
df_er = pd.DataFrame.from_dict(dados_coletados['ER'])
print('Resultados Experimentos em Rede Erdos Renyi')
display(df_er)
df_ws = pd.DataFrame.from_dict(dados_coletados['WS'])
print('Resultados Experimentos em Rede Watts Strogatz')
display(df_ws)

In [ ]:
# Adição de labels para os tipos de rede:
df_ba['tipo_rede'] = 'BA'
df_er['tipo_rede'] = 'ER'
df_ws['tipo_rede'] = 'WS'

# Junção dos dados em um único dataframe:
df_juncao = pd.concat([df_ba, df_er, df_ws], ignore_index=True)
display(df_juncao)

In [ ]:
# Normalização dos dados:
dados_normalizados = StandardScaler().fit_transform(df_juncao.loc[:, list(df_juncao.columns)[:-1]])

# Cálculo do PCA com 2 componentes:
pca = PCA(n_components=2)
componentes_principais = pca.fit_transform(dados_normalizados)

# Cálculo da variância explicada por componente principal:
print('Variância por Componente Principal:', pca.explained_variance_ratio_)

# Criação de DataFrame com o resultado:
df_cps = pd.DataFrame(componentes_principais, columns=['CP1', 'CP2'])
display(df_cps)

In [ ]:
# Mostragem do resultado graficamente:
tipos = ['BA', 'ER', 'WS']
plt.figure(figsize=(10, 6))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Primeira Componente Principal')
plt.ylabel('Segunda Componente Principal')
plt.title('Análise PCA de Medidas de Redes de Diferentes Topologias')
for tipo in tipos:
    indices_to_keep = df_juncao['tipo_rede'] == tipo
    plt.scatter(df_cps.loc[indices_to_keep, 'CP1'], df_cps.loc[indices_to_keep, 'CP2'], ec='black', s=50)
plt.legend(tipos, loc=(1.01, 0));
